In [4]:
from langfuse import Langfuse
from fsm_core.llm_common import get_sync_client
from compiler.core import Compiler
from application import Application

In [5]:
langfuse_client = Langfuse()
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
m_client = get_sync_client("bedrock", client_params={"aws_profile":"dev", "aws_region": "us-west-2"})

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [6]:
from application import InteractionMode
builder = Application(m_client, compiler, interaction_mode=InteractionMode.INTERACTIVE)

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [7]:
prepare_res = builder.prepare_bot(["bot that greets me by name"], bot_id="1337_420", capabilities=[])

In [8]:
prepare_res

ApplicationPrepareOut(refined_description=RefineOut(refined_description='', error_output=None), capabilities=CapabilitiesOut(capabilities=[], error_output=None), typespec=TypespecOut(reasoning="Based on the user's request, they want a bot that can greet them by name. This is a simple greeting bot that needs to:\n1. Store a user's name\n2. Greet the user with their name\n3. Update the user's name if they want to change it\n\nSince TypeSpec needs appropriate models and interfaces, I'll create:\n- A model for user information (name)\n- Functions to:\n  - Set/update the user's name\n  - Get a greeting using the stored name\n  - Reset the user's information\n\nThe scenarios would cover:\n- Initial greeting with no name set\n- Setting a name\n- Greeting with a stored name\n- Updating an existing name\n- Resetting user information", typespec_definitions='model UserInfo {\n  name: string;\n}\n\nmodel GreetingOptions {\n  timeOfDay?: string;\n  formal?: boolean;\n}\n\nmodel EmptyRequest {\n}\n\

In [9]:
prepare_res.typespec.typespec_definitions

'model UserInfo {\n  name: string;\n}\n\nmodel GreetingOptions {\n  timeOfDay?: string;\n  formal?: boolean;\n}\n\nmodel EmptyRequest {\n}\n\ninterface GreetingBot {\n  @scenario(\n  """\n  Scenario: Setting a user\'s name\n  When the user says "My name is John"\n  Then system should store the name "John"\n  And respond with a confirmation\n  \n  Examples:\n  | Input                     | Expected Name |\n  | "Call me Sarah"           | "Sarah"       |\n  | "I\'m Michael"             | "Michael"     |\n  | "Please call me Dr. Lee"  | "Dr. Lee"     |\n  """)\n  @llm_func("Store the user\'s name")\n  setUserName(options: UserInfo): void;\n\n  @scenario(\n  """\n  Scenario: Greeting a user with stored name\n  Given the system has stored name "Alex"\n  When the user says "Hello" or "Hi"\n  Then system should respond with a greeting including "Alex"\n\n  Scenario: Greeting a user at specific time of day\n  Given the system has stored name "Alex"\n  When the user says "Good morning"\n  Then 

In [10]:
reasoning = prepare_res.typespec.reasoning
typespec_schema = prepare_res.typespec.typespec_definitions
fake_msg = f"<reasoning>{reasoning}</reasoning><typespec>{typespec_schema}</typespec>"

fake_msg

'<reasoning>Based on the user\'s request, they want a bot that can greet them by name. This is a simple greeting bot that needs to:\n1. Store a user\'s name\n2. Greet the user with their name\n3. Update the user\'s name if they want to change it\n\nSince TypeSpec needs appropriate models and interfaces, I\'ll create:\n- A model for user information (name)\n- Functions to:\n  - Set/update the user\'s name\n  - Get a greeting using the stored name\n  - Reset the user\'s information\n\nThe scenarios would cover:\n- Initial greeting with no name set\n- Setting a name\n- Greeting with a stored name\n- Updating an existing name\n- Resetting user information</reasoning><typespec>model UserInfo {\n  name: string;\n}\n\nmodel GreetingOptions {\n  timeOfDay?: string;\n  formal?: boolean;\n}\n\nmodel EmptyRequest {\n}\n\ninterface GreetingBot {\n  @scenario(\n  """\n  Scenario: Setting a user\'s name\n  When the user says "My name is John"\n  Then system should store the name "John"\n  And respon

In [12]:
builder = Application(m_client, compiler)

final_res = builder.update_bot(
    typespec_schema=fake_msg,
    bot_id="1337_420",
    capabilities=[],
)

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [9]:
final_res

ApplicationOut(refined_description=RefineOut(refined_description='', error_output=None), capabilities=CapabilitiesOut(capabilities=[], error_output=None), typespec=TypespecOut(reasoning="The user requested a bot that greets them by name. This is a simple use case where the bot needs to:\n1. Store and retrieve the user's name\n2. Generate appropriate greetings based on the user's name\n3. Potentially support different greeting styles or times of day\n\nFor this bot, we need:\n- A model to represent the user's information (name)\n- A model for greeting options (time of day, style)\n- Functions to set the user's name and to generate greetings\n\nThe interface should support:\n- Setting the user's name\n- Getting a greeting with the user's name\n- Possibly updating greeting preferences", typespec_definitions='model UserInfo {\n  name: string;\n}\n\nmodel GreetingOptions {\n  timeOfDay: string;\n  style: string;\n}\n\nmodel UpdateNameOptions {\n  name: string;\n}\n\ninterface GreetingBot {\

In [14]:
from core.interpolator import Interpolator

import os

# Create the botbuild directory if it doesn't exist
if not os.path.exists("../botbuild"):
    os.makedirs("../botbuild")

interpolator = Interpolator(".")

interpolator.bake(final_res, "../../bot_result")
